# CE: "Kanselering "Da da da daaaa" i Beethovens 5. bomb av en sang.".
### Av B-dog og Siggen.

In [ ]:
# Nødvendige importer
from matplotlib import style
style.use("default") # Gir finere plot for .ipynb filer i VS Code
import matplotlib.pyplot as plt
import numpy as np
from numba import jit
from scipy.io import wavfile
import warnings
warnings.filterwarnings("ignore") # Ignorerer advarsler fra numba

### Wavelet analyse av lydsignalet:
Planen er å kjøre en wavelet analyse på lydfilen av sangen og manuelt identifisere hvilke frekvenser som dukker opp når, for så å kunne lage mot frekvensen til hver bølge i lydsignalet. 

### Lydbølge:
Vi begynner med å lage oss en test bølge med to forskjellig frekvenser i forskjellige tidspunkt. Det vi bruker her er 1 Hz frem til 5 sekunder og 0.5 Hz til 10 sekunder. Vi bruker en klassisk cosinus bølge:
$$y(x)=A\cos(\omega x + \phi)$$
Vi ser her også at vi må faseforskyve den andre delen av bølgen slik at den henger pent sammen. Koden for dette ser slik ut:

In [ ]:
#Definerer variabeler:
omega = 2*np.pi #Vinkelfrekvens
A = 1 #Amplitude
phi = 0 #Forskyvningsledd
x = np.linspace(0,10,1000) #tidsarray
#Første bølge:
y = A*np.cos(omega*x + phi)
#Andre bølge:
for i in range(500):
    y[i+500] = A*np.cos(0.5*omega*x[i+500] + np.pi) #Annen frekvens etter indeks 500
#plot:
plt.Figure(figsize=(16,16)) #justerer plottstørrelsen
plt.plot(x,y,label="lydbølge på 1 Hz") #plotter bølgen
plt.xlabel("t [s]") #x-akse tittel
plt.ylabel("y(x)") #y-akse tittel
plt.legend() #viser navn på graf

Her ser vi da altså en lydbølge på 1 Hz frem til 5 sekunder og 0.5 Hz til 10 sekunder.

### Fourieranalyse:
For å identifisere disse frekvensene fra lydbølgen starter vi med Fourieranalyse. Det er et matematisk verktøy som lar oss gå fra tid på x-aksen til frekvensaksen. Matematisk ser det slik ut:
$$F(\omega) = \frac{1}{2\pi}\int_{-\infty}^{\infty}f(t)e^{i\omega x}dt$$
Vi bruker numpys Forier transformasjon siden vi vil gjøre dette numerisk. Det numpy gjør er kort forklart at den sjekker hvordan høyden går over tid på en bølge med en kjent samplingsfrekvens som vi har lært at skal være: 
$$f_s<2f$$
Hvor fs er samplingsfrekvensen og f er frekvensen til signalet. Dette kalles Nyquist frekvensen og den må oppfylles for at fourieranalysen skal gi rett frekvenser og ikke missforstå signalet til å ha andre frekvenser enn det egentlig har.

In [ ]:
plt.Figure(figsize=(16,16))#justerer figurstørrelsen
plt.plot(x,y,label="lydbølge på 1 Hz") #plotter bølgen
fs = 3 #hvor ofte prikkene i plottet skal komme
plt.plot(x[::fs],y[::fs],"o",label="samplingspunkter") #plotter prikker på grafen som skal være samplingen til numpy
plt.xlabel("t [s]") #x-akse tittel
plt.ylabel("y(x)") #y-akse tittel
plt.legend() #viser graf navn

Vi ser her hvordan prikkene på grafen indikerer hvordan numpys Fourier sampling skjer.

Når vi bruker numpy lager vi en funksjon som vi kan bruke på et hvert signal. Vi plotter både signalet og Fourieranalysen av signalet med denne koden:

In [ ]:
@jit
def Fourier(data,fs):
    ## Funksjon som tar inn et data signal (lydfil) og en samplingsfrekvens og gjør en forier analyse av signalet og plotter signalet i tids og frekvens-rommet
    x_n  = data # Dataserien egentlig bytter bare navn
    N = int(len(x_n)) # Antall punkter i signalet
    
    T = N / fs # tital tid for en tidsarray til forertransformasjonen
    dt = 1 / fs # lengden på hvert tidssteg
    t = dt*np.linspace(0, N-1, N) #lager tidssarray
 
    
    X_k = (1/N)*np.fft.fft(x_n) #kjører forier transformasjoenen på signalet
    freq = np.fft.fftfreq(N, dt) #genrerer frekvens domenet til transformasjonen
    #plotter signalet:
    plt.subplot(2,1,1) #generer første subplot
    plt.plot(t, x_n,label="Signal i tidssrommet") #Plotter signalet i tidsrommet
    plt.xlabel('t [s]') #x-akse tittel 
    plt.ylabel('x$_n$') #y-akse tittel
    plt.legend() #Viser graf navn
    #plotter Analysen
    plt.subplot(2,1,2) #generer andre subplot
    plt.plot(freq, np.abs(X_k),label="signalet i frekvensrommet") #Plotter signalet i frekvensrommet
    plt.xlabel('frekvens [Hz]') #x-akse tittel
    plt.ylabel('X$_k$') #y-akse tittel
    plt.legend() #viser grafnavn
    plt.xlim([0, 5]) #begrenser hva som vises på x aksen
    plt.show() #viser plot

In [ ]:
fs = 100 #setter samplingsfrekvensen
Fourier(y,fs) #bruker funksjonen på det genererte signalet

Her ser vi signalet og analysen. Vi ser at vi klarer å idenifisere to frekvenser med de to toppene i plott nr to, men de er ikke så skarpe som vi helst skulle hadt de. Dette kan skylles at en Fourier analyse funker på et signal som består av hele signaler og ikke frekvenser som endrer seg over tid. Dette er grunnen til at vi går over til Waveletanalyse.

### Waveletanalyse:
Denne formen av lydsampling er noe lik Fourier analyse 
$$\gamma (\omega _a,t)$$

In [ ]:
@jit
def psi(t,wa,tk,K,fs): # Likning 14.8 i Vistnes
    ## Funksjon som generer Waveleten som skal multipliseres med signalet
    # Disse tre er bare deler av en større funksjon som legges til tilslutt:
    w = np.exp(-1j*wa*(t-tk)) 
    k = np.exp(-K**2)
    gaus = np.exp(-(wa**2 * (t-tk)**2)/(2*K)**2)
    C=0.798*wa/(fs*K) # Ligning 14.7 i Vistnes
    return C*(w-k)*gaus #returnerer waveleten

In [ ]:
@jit
def gamma(xn,t,w,K,fs): # Likning 14.9 i Vistnes
    ## Funksjon som kjører en versjon av alle frekvenser til alle tidspunkter og summerer produktet av signalet og waveleten og lager en matrise til verdiene
    N = int(len(w)) #Henter lengden på frekvensarrayet
    T = int(len(t)) #Henter lengden på tidsarrayet
    gamma = np.zeros((N,T),dtype=np.complex_) #genererer en tom matrise
    for i in range(N): #løper gjennom alle frekvenser
        for ii in range(T): #løper gjennom alle tidspunkter
            psi1 = psi(t,w[i],t[ii],K,fs) #genererer den gjeldene waveleten
            gamma[i,ii] = np.sum(xn*np.conjugate(psi1)) #summerer opp produktet av signalet og waveleten
    return gamma #returnerer matrisen

In [ ]:
@jit
def wavelet(data, t, w, K, fs):
    ## Kjører gamma funksjonen og genererer et contour plot med absolutverdien av matrisen siden den blir kompleks
    grid = gamma(data,t,w,K,fs) #genererer matrisen
    plt.contourf(t,w,np.abs(grid)) # plotter griddet med frekvensene og tidspunktene
    plt.colorbar() #viser hvilke verdier fargene har
    plt.xlabel("tid [s]") #x-akse tittel
    plt.ylabel("frekvens [log10(2*pi*HZ)]") #y-aksetittel
    plt.show() #viser plottet
    return grid # Returnerer gamma-griddet

In [ ]:
K = 4 #setter K verdien
N = 1000 # definerer hvor mange 
w = np.linspace(0,10,N) #Frekvensarray
t = np.linspace(0,10,1000) #tidsarray
fs = 100 #samplingsfrekvens
wavelet_grid = wavelet(y, t, w, K, fs) # kjører plottefunksjonen

# FFT av DDD

In [ ]:
@jit
def readWav(filename):
    ## Leser lydfilen med navnet filename, og returnerer tidpunkter t, signal x_n og samplingsfrekvens fs
    fs, data = wavfile.read(filename) # Henter samplingsfrekvens og signaldata fra lydfilen
    x_n = data[:,0] # Henter ut relevant del av signalet
    N = len(x_n) # lengden på lyd signalet
    T = N/fs #lengden på tidsarrayen
    t = np.linspace(0, T, N) # Lager en array med tidpunkter
    return t, x_n, fs # returnerer tidsarrayet, lydsignalet og samplingsfrekvensen

In [ ]:
@jit
def fourier(t, x_n, N):
    ## Gjør en Fourier-analyse på signal x_n med tidpunkter t. N er lengden vi ønsker til arrayet X_k fra fft.
    ## Returnerer frekvenser freq og utslag X_k fra fft.
    X_k = 1/N * np.fft.fft(x_n, n=N) # Gjør en fft på x_n. Deler på N for å korrigere størrelsen på utslagene.
    dt = t[1] - t[0] #lengden på hvert tidssteg
    freq = np.fft.fftfreq(N, dt) # Finner frekvensområdet til spekteret fra analysen.
    return freq, X_k #returnerer frekvensdomenet og fourier transformasjonen

In [ ]:
lydfil_lu = "liteUtdrag.wav" # Lydfilen med et 30s utdrag av symfonien
t_lu, x_n_lu, fs_lu = readWav(lydfil_lu) # Henter data fra lydfilen
N_lu = len(x_n_lu) # Finner lengden til signalet (antall datapunkter)
freq_lu, X_k_lu = fourier(t_lu, x_n_lu, N_lu) # Gjør en Fourieranalyse på utdraget

In [ ]:
lydfil_ddd = "DDD.wav" # Lydfilen med kun "DUN DUN DUN DUUUN"-delen av symfonien.
t_ddd, x_n_ddd, fs_ddd = readWav(lydfil_ddd) # Henter data fra lydfilen.
freq_ddd, X_k_ddd = fourier(t_ddd, x_n_ddd, N_lu) # Gjør en Fourieranalyse på DDD. Bruker N_lu for å få lik lengde på X_k_ddd og X_k_lu

In [ ]:
## Plotter lydsignalet fra DDD
plt.title(f"Plot av lydfilen {lydfil_ddd}")
plt.xlabel("$t$ [s]")
plt.ylabel("$A$")
plt.plot(t_ddd, x_n_ddd)

In [ ]:
## Plotter frekvensspekteret til DDD
plt.xlim(0, fs_ddd/2) # Største mulige frekvens må være halvparten av samplingsfrekvensen (Nyquist)
plt.title(f"Fouriertransformasjon av {lydfil_ddd}")
plt.xlabel("$f$ [Hz]")
plt.ylabel("A")
plt.plot(freq_ddd, X_k_ddd, label=f"DFT av {lydfil_ddd}")
plt.legend()

In [ ]:
## Zoomer inn på plottet av frekvensspekteret til DDD til relevante verdier
plt.xlim(0, 2000)
plt.title(f"Fouriertransformasjon av {lydfil_ddd}")
plt.xlabel("$f$ [Hz]")
plt.ylabel("A")
plt.plot(freq_ddd, X_k_ddd, label=f"DFT av {lydfil_ddd}")

In [ ]:
## Plotter lydsignalet fra utdraget på 30s
plt.plot(t_lu, x_n_lu)

In [ ]:
## Plotter frekvensspekteret til utdraget
plt.xlim(0, 1000)
plt.title(f"Fouriertransformasjon av {lydfil_lu}")
plt.xlabel("$f$ [Hz]")
plt.ylabel("A")
plt.plot(freq_lu, X_k_lu, label=f"DFT av {lydfil_lu}")

In [ ]:
X_k_ny = X_k_lu - X_k_ddd # Fjerner frekvensene til DDD fra utdraget

## Plotter frekvensspekteret til DDD, utdraget, og differansen
plt.xlim(0,1000)
plt.plot(freq_ddd, X_k_ddd, "r:")
plt.plot(freq_lu, X_k_lu, "b:")
plt.plot(freq_lu, X_k_ny, "g")

In [ ]:
x_n_ny = np.fft.ifft(X_k_ny) # Invers Fouriertransformasjon, for å få tilbake lydsignalet uten DDD

## Plotter det nye lydsignalet
plt.plot(t_lu, x_n_ny)

In [ ]:
wavfile.write("bby.wav", fs_lu, np.real(x_n_ny)) # Skriver lydsignalet ut til ny lydfil

# Wavelet av DDD

In [ ]:
@jit
def sliced_wavelet(t, x_n, freq, jump, max_freq_index):
    ## Lager sliced versjoner av t, x_n og frekvensene for at wavelet() skal kjøre raskere
    t_sliced = t[::jump]
    x_n_sliced = x_n[::jump]
    freq_sliced = freq[0:max_freq_index:jump]
    return t_sliced, x_n_sliced, freq_sliced # returnerer sliced arrayer

In [ ]:
K = 20 #setter K verdien
slice_jump = 100
max_freq_index = 40000
t_ddd_sliced, x_n_ddd_sliced, freq_ddd_sliced = sliced_wavelet(t_ddd, x_n_ddd, freq_ddd, slice_jump, max_freq_index)

print(len(t_ddd_sliced))
print(len(x_n_ddd_sliced))
print(len(freq_ddd_sliced))

In [ ]:
grid_ddd = wavelet(x_n_ddd_sliced, t_ddd_sliced, freq_ddd_sliced, K, fs_ddd)

In [ ]:
t_lu_sliced, x_n_lu_sliced, freq_lu_sliced = sliced_wavelet(t_lu, x_n_lu, freq_lu, slice_jump, max_freq_index)


In [ ]:
grid_lu = wavelet(x_n_lu_sliced, t_lu_sliced, freq_lu_sliced, K, fs_lu)

In [ ]:
def analyze_wavelet(grid_big, grid_small, tol):
    Nw_big, Nt_big = np.shape(grid_big)
    Nw_small, Nt_small = np.shape(grid_small)
    N = Nt_big + 2 - Nt_small # Antall startverdier vi ser på for t
    indices = np.array([])
    for i in range(Nt_big-Nt_small+1):
        grid_i = grid_big[:, i:i+Nt_small]
        if np.allclose(grid_small, grid_i, atol=tol):
            indices = np.append(indices, i)
    return indices

In [ ]:
indices = analyze_wavelet(grid_lu, grid_ddd, 5)
indices = indices * 100
Nw_small, Nt_small = np.shape(grid_ddd)
Nt_small = Nt_small * 100
x_n_new = np.copy(x_n_lu)

In [ ]:
print(indices)
for index in indices:
    index = int(index)
    x_n_new[index:index+Nt_small] = np.zeros(Nt_small)

In [ ]:
wavfile.write("Woopwoop.wav", fs_lu, np.real(x_n_new))

# Waveletanalyse å